In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
%matplotlib inline

In [5]:
solar_data = pd.read_csv('data/train/train.csv',encoding='utf-8')
solar_data.drop(['Day', 'Hour','Minute'], axis='columns', inplace=True)

In [6]:
def create_dataset(solar_data, index, input_days=7, output_days=2):
    dataX, dataY = [], []
    for i in range(0,48*7):
        dataX += list(np.array(solar_data.loc[index+i].tolist()))
    for i in range(48*7,48*(7+2)):
        dataY += solar_data.loc[[0+i],['TARGET']].values.tolist()
    return np.array(dataX), np.array(dataY)

In [9]:
# 데이터셋 생성
input_data, output_data = [], []
last_index = 3*365*48-48*(7+2)
index_list = list(range(0,last_index+1,48))
for i in index_list:
    X, Y = create_dataset(solar_data,i)
    input_data.append(X)
    output_data.append(Y)

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(input_data)
input_data = scaler.transform(input_data)
input_data = input_data.reshape(-1, 178, 1)
input_data

array([[ 0.        ,  0.        , -0.55789236, ..., -1.14314168,
        -0.66496767,  0.        ],
       [ 0.        ,  0.        , -0.47333728, ...,  1.20048417,
        -1.43914103,  0.        ],
       [ 0.        ,  0.        ,  0.03399316, ...,  0.37385411,
        -0.92302546,  0.        ],
       ...,
       [ 0.        ,  0.        , -1.31888803, ..., -1.1449505 ,
        -0.79399656,  0.        ],
       [ 0.        ,  0.        , -1.0652228 , ..., -1.20102388,
        -0.79399656,  0.        ],
       [ 0.        ,  0.        ,  0.62587868, ..., -0.92789228,
        -0.92302546,  0.        ]])

In [16]:
input_data.shape

(1087, 2016)

In [13]:
# 데이터셋 분배
X_train, X_test, y_train, y_test = train_test_split(np.array(input_data), np.array(output_data), test_size = 0.1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

In [14]:
# 모델 구성
# model = Sequential()
# model.add(Dense(32,input_dim=2016,activation="relu"))
# model.add(Dropout(0.3))
# for i in range(2):
#     model.add(Dense(32,activation="relu"))
#     model.add(Dropout(0.3))
# model.add(Dense(1))

model = Sequential()
model.add(LSTM(64, dropout=0.2, input_shape=(2016, 1)))
model.add(Dense(1, activation='elu'))
model.compile(loss="mean_squared_error", optimizer = "RMSProp", metrics=["accuracy"])

In [15]:
# 모델 학습
hist = model.fit(X_train, y_train, epochs=200, batch_size=32,validation_data=(X_val, y_val))

Epoch 1/200


ValueError: in user code:

    C:\Users\uoo1325\Anaconda3\envs\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    C:\Users\uoo1325\Anaconda3\envs\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\uoo1325\Anaconda3\envs\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\uoo1325\Anaconda3\envs\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\uoo1325\Anaconda3\envs\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:531 train_step  **
        y_pred = self(x, training=True)
    C:\Users\uoo1325\Anaconda3\envs\python37\lib\site-pac